In [1]:
import os
import requests
import json

In [27]:
WB_TOKEN = os.getenv('WB_TOKEN')


def upload_single_image_to_wb(file_path, nm_id, api_key, photo_number=1):
  
    url = "https://content-api.wildberries.ru/content/v3/media/file"
    
    headers = {
        "Authorization": api_key,
        "X-Nm-Id": str(nm_id),
        "X-Photo-Number": str(photo_number)
    }
    
    with open(file_path, "rb") as f:
        files = {
            "uploadfile": (os.path.basename(file_path), f, "image/jpeg")
        }
        
        try:
            response = requests.post(url, headers=headers, files=files)
        except requests.RequestException as e:
            print(f"Ошибка при отправке запроса: {e}")
            return
        
        if response.status_code == 200:
            resp_json = response.json()
            if not resp_json.get("error", False):
                print(f"Успешно загружено изображение '{os.path.basename(file_path)}' на nmID={nm_id}.")
            else:
                print(f"Ошибка Wildberries при загрузке: {resp_json.get('errorText', 'Неизвестная ошибка')}")
        else:
            print(f"Ошибка ответа сервера. Код: {response.status_code}, файл: {os.path.basename(file_path)}")

def process_vendor_list(vendor_list, images_folder, api_key):
    """
    Проходит по списку vendor_list, проверяет, существует ли соответствующий файл в images_folder,
    и при наличии вызывает функцию загрузки на WB.
    
    :param vendor_list: Список словарей вида [{'nmID': ..., 'vendorCode': ' zip_lock_XXXX_YY'}, ...]
    :param images_folder: Путь к папке, где лежат файлы (например, 'images_900_1200').
    :param api_key: API-ключ для Wildberries.
    """
    for item in vendor_list:
        nm_id = item["nmID"]
        vendor_code = item["vendorCode"]
        
        # Ожидаем формат vendorCode = "zip_lock_9604677_30"
        # Удаляем " zip_lock_" в начале:
        # Неподходящий формат vendorCode: 'zip_lock_9604896_100'. Пропускаем.
        
        if vendor_code.startswith("zip_lock_"):
        
            code_part = vendor_code[9:]  # "9604677_30"
            # Разделяем по символу "_"
            parts = code_part.split("_", maxsplit=1)
            if len(parts) == 2:
                product_id, quantity_str = parts
                # Формируем имя файла: "9604677-30.jpg"
                image_name = f"{product_id}-{quantity_str}.jpg"
                file_path = os.path.join(images_folder, image_name)
                
                # Проверяем, существует ли файл
                if os.path.isfile(file_path):
                    print(f"Файл '{image_name}' найден. Загружаем на nmID={nm_id}...")
                    upload_single_image_to_wb(file_path, nm_id, api_key, photo_number=1)
                else:
                    print(f"Файл '{image_name}' не найден в папке '{images_folder}'. Пропускаем.")
            else:
                print(f"Неподходящий формат vendorCode: '{vendor_code}'. Пропускаем.")
        else:
            print(f"Неподходящий формат vendorCode: '{vendor_code}'. Пропускаем.")


In [29]:
all_cards_list = "https://content-api.wildberries.ru/content/v2/get/cards/list"
headers = {
    "Authorization": WB_TOKEN,
}
body =             {
          "settings": {                      
            "cursor": {
              "limit": 100
            },
            "filter": {
              "withPhoto": 0
            }
          }
        }


resp = requests.post(all_cards_list, headers=headers, json=body)

In [30]:
d = json.loads(resp.text)
d

{'cards': [{'nmID': 324346455,
   'imtID': 305703882,
   'nmUUID': '0194c36b-07bd-7deb-b574-b61da75695de',
   'subjectID': 3756,
   'subjectName': 'Пакеты упаковочные',
   'vendorCode': 'zip_lock_9601412_100',
   'brand': '',
   'title': 'Пакеты Zip Lock с ручкой зеленый металлик 350*350 мм 100 шт',
   'description': 'Преимущества:\nВместительный размер 35*35 см - можно вложить документы формата А4;\nНаличие прорубных ручек;\nНепрозрачный цвет (зеленый металлик) скроет от посторонних глаз содержимое пакета.',
   'needKiz': False,
   'dimensions': {'width': 35, 'height': 2, 'length': 35, 'isValid': True},
   'sizes': [{'chrtID': 487209177,
     'techSize': '0',
     'wbSize': '',
     'skus': ['2042713193523']}],
   'createdAt': '2025-02-01T21:32:35.745451Z',
   'updatedAt': '2025-02-01T21:39:25.366356Z'},
  {'nmID': 324346459,
   'imtID': 305703885,
   'nmUUID': '0194c36b-02ea-78d0-b53e-82557596d07c',
   'subjectID': 3756,
   'subjectName': 'Пакеты упаковочные',
   'vendorCode': 'zip_l

In [31]:
newNmIdVendorCodes = [{"nmID": row["nmID"], "vendorCode": row["vendorCode"]} for row in d["cards"]]

In [32]:
newNmIdVendorCodes

[{'nmID': 324346455, 'vendorCode': 'zip_lock_9601412_100'},
 {'nmID': 324346459, 'vendorCode': 'zip_lock_9601411_100'},
 {'nmID': 324346461, 'vendorCode': 'zip_lock_9604019_100'}]

In [33]:
process_vendor_list(newNmIdVendorCodes, "./zip_locks_images_900_1200", WB_TOKEN)

Файл '9601412-100.jpg' найден. Загружаем на nmID=324346455...
Успешно загружено изображение '9601412-100.jpg' на nmID=324346455.
Файл '9601411-100.jpg' найден. Загружаем на nmID=324346459...
Успешно загружено изображение '9601411-100.jpg' на nmID=324346459.
Файл '9604019-100.jpg' найден. Загружаем на nmID=324346461...
Успешно загружено изображение '9604019-100.jpg' на nmID=324346461.
